In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, run_parallel_stitch
from labelbox import Project, Dataset, Client
import os
from amftrack.transfer.functions.transfer import upload, zip_file
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
import uuid
import json
import numpy as np

PROJECT_ID = "cl0v5xcu33a330z5mhjx7bgy4"
DATASET_ID = "cl0v4z5373b0w0z8v9t1x08jd"
PROJECT_NAME = "Fungi"
DATASET_NAME = "Fungal network"
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"
API_KEY = os.getenv("LABELBOX_API_KEY")
client = Client(api_key=API_KEY)
project = client.get_project(PROJECT_ID)
dataset = client.get_dataset(DATASET_ID)
directory = directory_project
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']=="94"]
i=80
folder_list = list(folders['folder'])
folder_list.sort()
directory_name = folder_list[i]
path_snap=directory+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)

/home/cbisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


AuthenticationError: Labelbox API key not provided('Labelbox API key not provided', None)

In [23]:
project = client.create_project(name="fungi_new2")
dataset = client.create_dataset(name="image_annotation_import_demo_dataset")

In [7]:
!pip install imagesize

In [24]:
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.LINE, name="Line")
])
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

project.setup(editor, ontology_builder.asdict())
project.datasets.connect(dataset)


In [32]:
def create_objects():
    line = Line(points=[Point(x=60,y=70), Point(x=65,y=100), Point(x=80,y=130), Point(x=40,y=200)])
    line_annotation = ObjectAnnotation(value=line, name="Line")
    return line_annotation


In [38]:
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)

In [39]:
image_data = ImageData(uid=data_row.uid)
line_annotation = create_objects()
label = Label(
    data=image_data,
    annotations = [line_annotation
    ]
)

In [40]:
def signing_function(obj_bytes: bytes) -> str:
    url = client.upload_data(content=obj_bytes, sign=True)
    return url


label.add_url_to_masks(signing_function)

Label(uid=None, data=ImageData(im_bytes=None,file_path=None,url=None,arr=None), annotations=[ObjectAnnotation(name='Line', feature_schema_id=None, extra={}, value=Line(extra={}, points=[Point(extra={}, x=60.0, y=70.0), Point(extra={}, x=65.0, y=100.0), Point(extra={}, x=80.0, y=130.0), Point(extra={}, x=40.0, y=200.0)]), classifications=[])], extra={})

In [42]:
mal_label = Label(
    data=image_data,
    annotations = [line_annotation
    ]
)

label.add_url_to_masks(signing_function)

mal_label.assign_feature_schema_ids(ontology_builder.from_project(project))

ndjson_labels = list(NDJsonConverter.serialize([mal_label]))
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="upload_label_import_job2", 
    predictions=ndjson_labels)